# MATH 210 Introduction to Mathematical Computing

## March 1, 2017

* Approximating definite integrals with `scipy.integrate`
    * Trapezoid rule: `trapz`
    * Simpson's rule: `simps`
    * QUADPACK: `quad`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Approximating definite integrals with `scipy.integrate`

The package `scipy.integrate` has several functions for computing integrals and solving differential equations. We will start with the functions `trapz`, `simps` and `quad` for computing definite integrals. Check out the [documentation](https://docs.scipy.org/doc/scipy-0.18.1/reference/integrate.html).

In [ ]:
import scipy.integrate as spi

### Example: Trapezoid rule

Let's use `trapz` to compute some simple integrals for which we know the exact value:

$$
\int_0^{\pi} \sin(x) \, dx = 2
$$

$$
\int_{-1}^1 \sqrt{1 - x^2} \, dx = \frac{\pi}{2}
$$

$$
\int_0^1 x e^{-x^2} dx = \frac{1}{2} \left( 1 - \frac{1}{e} \right)
$$

First, let's consult the documentation for `trapz`:

In [ ]:
spi.trapz?

We see that all we need to do it supply arrays of $x$ and $y$ values for the integrand and `trapz` returns the approximation of the integral using the [trapezoid rule](https://en.wikipedia.org/wiki/Trapezoidal_rule). The number of points we give to `trapz` is up to us but we have to remember that more points gives a better approximation but it takes more time to compute!

In [ ]:
x = np.linspace(0,np.pi,1000)
y = np.sin(x)
spi.trapz(y,x) # Integral is exactly 2

In [ ]:
x = np.linspace(-1,1,1000)
y = np.sqrt(1 - x**2)
spi.trapz(y,x) # Integral is exactly pi/2

In [ ]:
np.pi/2

In [ ]:
x = np.linspace(0,1,1000)
y = x * np.exp(-x**2)
spi.trapz(y,x) # Integral is exactly (1 - e^{-1})/2

In [ ]:
(1 - np.exp(-1))/2

### Example: Simpson's rule

[Simpson's rule](https://en.wikipedia.org/wiki/Simpson%27s_rule) is based on the idea that we can approximate the function $f(x)$ by a different quadratic polynomial on each subinterval of a partition. This is a slight improvement over the trapezoid rule which approximate $f(x)$ by a straight line on each subinterval of a partition. And again we need to keep in mind that the number of subintervals in the partition is up to us and more points gives a better approximation but requires more computing time!

Let's use `simps` to compute the following integral

$$
\int_0^1 \left( \frac{x^{n-1}}{1 - x^{1/p}} - \frac{px^{np-1}}{1-x} \right) dx = p \ln p \ \ , \ p > 0
$$

Notice that the integrand is not defined at $x=1$ and so we can modify the interval $[0,1]$ to be $[0,1 - \epsilon]$ for some small nonzero $\epsilon$.

Let's look at the documentation for `simps`:

In [ ]:
spi.simps?

The function works in the same way as `trapz` and so we only need to create arrays of $x$ and $y$ values.

In [ ]:
n = 10
p = 3
N = 1000
epsilon = 0.0000001

x = np.linspace(0,1 - epsilon,N)
y = x**(n-1)/(1 - x**(1/p)) - p*x**(n*p - 1)/(1 - x)

spi.simps(y,x)

In [ ]:
p*np.log(p)

Let's compare the `simps` result to `trapz`:

In [ ]:
np.abs(spi.simps(y,x) - p*np.log(p)) < np.abs(spi.trapz(y,x) - p*np.log(p))

Simpson's rule gives a better approximation as expected!

### Example: Trapezoid rule versus Simpson's rule

Simpson's rule will usually give a better estimate than the trapezoid rule. So why even use the trapezoid rule? Because it does fewer computations than Simpson's rule given the same points and so it's faster! Check it out!

In [ ]:
N = 1000000
x = np.linspace(0,np.pi,N)
y = np.sin(x)

In [ ]:
%%timeit
spi.trapz(y,x)

In [ ]:
%%timeit
spi.simps(y,x)

The trapezoid rule is 4 times faster!!!

### Example: QUADPACK

The function `quad` uses a technique from the [QUADPACK](https://en.wikipedia.org/wiki/QUADPACK) library. This is by far the best tool in SciPy for approximating definite integrals but the algorithm is beyond the scope of this course and so we will use it as a blackbox. Let's look at the documentation for `quad`:

In [ ]:
spi.quad?

The function `quad` works a bit differently that `trapz` and `simps`. Instead of giving arrays of $x$ and $y$ values of the function $f(x)$ that we want to integrate, we give the function $f(x)$ itself along with the limits of integration $a$ and $b$. The function `quad` returns a tuple `(I,abserr)` where `I` is an approximation of the integral

$$
\mathtt{I} \approx \int_a^b f(x) \, dx
$$

and `abserr` is an estimate for the error in the approximation

$$
\left| \ I - \int_a^b f(x) \, dx \ \right| < \mathtt{abserr}
$$

Let's compute the integral using `quad`

$$
\int_0^{\infty} \frac{e^{-qx}}{\sqrt{x}} dx = \sqrt{\frac{\pi}{q}} \ \ , \ q > 0
$$

Notice that the integral is infinite and the integrand is not defined at $x=0$.

In [ ]:
q = 10 # Arbitrary choice for the parameter q
def f(x):
    return np.exp(-q*x)/np.sqrt(x)

I, abserr = spi.quad(f,0,np.inf) # Unpack the output into 2 variables I and error
print(I)

In [ ]:
np.sqrt(np.pi/q)

Let's verify that the error in the approximation is indeed less than the error estimate `error` returned by `quad`:

In [ ]:
abs(I - np.sqrt(np.pi/q)) < abserr

## Exercises

**Exercise 1.** Use both `trapz` and `simps` (with the same $x$ and $y$ values) to compute the following integrals:

1. $\int_0^3 2x^2 \sqrt{1 + x^3} \, dx$
2. $\int_0^1 x \sin(2 \pi x) \, dx$
3. $\int_0^1 \frac{1}{1 + x^2} dx$

Compute these integrals by hand (using substitution, integration by parts, etc.) and compare the exact values to the approximations.

**Exercise 2.** Verify the integral formula

$$
\int_0^1 \ln(x) \ln(1-x) \, dx = 2 - \frac{\pi^2}{6}
$$

**Exercise 3.** Verify the integral formula

$$
\int_0^{\pi/2} \frac{x^3 \cos(x)}{\sin^3(x)} dx = \frac{3}{2} \pi \ln(2) - \frac{\pi^3}{16}
$$